# Unit 2 - KNN Implementation 

---


Universidad Politécnica de Yucatán


Robotics 9B


Marco Alejandro González Barbudo


## I.	KNN Algorithm

K-Nearest Neighbors (KNN) is a machine learning algorithm used for classification and regression tasks. It operates on the idea that similar data points tend to belong to the same category or have similar values. KNN is a non-parametric and lazy learner algorithm, which means it doesn't make assumptions about the underlying data distribution and postpones the actual learning until a prediction is needed.

**How KNN Works:**
1. **Select the Number of Neighbors (K):** You start by choosing the number of nearest neighbors, denoted as "K." This value determines how many neighboring data points will be considered when making a prediction.

2. **Calculate Euclidean Distance:** The algorithm calculates the Euclidean distance between the new data point and all other data points in the dataset. Euclidean distance is like measuring the straight-line distance between two points in space.

3. **Select K Nearest Neighbors:** The KNN algorithm then selects the K data points with the shortest Euclidean distances to the new data point. Among the K nearest neighbors, the algorithm counts how many data points belong to each category or class.

4. **Assign the New Data Point:** The algorithm assigns the category of the new data point based on which category has the most neighbors among the K nearest neighbors.

5. **Model is Ready:** Your KNN model is now ready to make predictions for new data points.

**Choosing the Value of K:**
- There is no strict rule for selecting the best K value, so you often need to experiment with different values.
- A commonly used value for K is 5, but it can vary depending on your dataset.
- Very low K values (e.g., 1 or 2) can be sensitive to outliers and lead to noisy predictions.
- Large K values can provide smoother predictions but might not capture local patterns well.
- It is important to use odd values for K.

**Advantages of KNN:**
- Simple to implement and understand.
- Robust to noisy data.
- Effective when the training dataset is large.

**Disadvantages of KNN:**
- Requires selecting the appropriate K value.
- Computationally expensive for large datasets due to distance calculations for all training samples.

**Applications of KNN:**
- Data preprocessing: KNN can be used to impute missing values in datasets.
- Recommendation systems: It's used to suggest items or content based on user behavior.
- Finance: Assessing credit risk by predicting loan solvency.
- Healthcare: Predicting health-related outcomes like disease risk.
- Pattern recognition: Identifying patterns in text, images, and more.

**Strengths and Weaknesses:**
- Strengths: Easy to implement, adaptable to changing data, and requires few hyperparameters.
- Weaknesses: Inefficient for large datasets, sensitive to the curse of dimensionality (works less well in high-dimensional spaces), and prone to overfitting or underfitting depending on the choice of K.

## II.	Algorithm Pseudocode

START

&nbsp; &nbsp; &nbsp; INPUT: $D = {(X_1, c_1),...,(X_N,C_N)}$

$X = (x_1,...,x_n)$ new classification case
    
FOR all classified object $(x_i,...c_i)$

&nbsp; &nbsp; &nbsp; GET $d_i = d(X_i,X)$
        
SORT $d_i(i = 1,...,N)$ ascending

WHEN $K$ cases $D_X^K$ already classifed closer to $X$

ASSIGN to $X$ most frecuent class in $D_X^K$

END

## III. Algorithm Implementation


### Dataset Description

The data consists of 100,000 observations of space taken by the SDSS (Sloan Digital Sky Survey). Every observation is described by 17 feature columns and 1 class column which identifies it to be either a star, galaxy or quasar.

In [1]:
# Notebook Initialization
import numpy as np
import pandas as pd

# Loading Dataset
df = pd.read_csv("C:/Users/marco/Documents/Machine_Learning/Dataset/Stars.csv")
df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [2]:
# Preparing Dataset
df = df[[col for col in df.columns if col != 'class'] + ['class']]
df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,redshift,plate,MJD,fiber_ID,class
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,0.634794,5812,56354,171,GALAXY
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,0.779136,10445,58158,427,GALAXY
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,0.644195,4576,55592,299,GALAXY
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,0.932346,9149,58039,775,GALAXY
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,0.116123,6121,56187,842,GALAXY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,0.000000,9374,57749,438,GALAXY
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,0.404895,7626,56934,866,GALAXY
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,0.143366,2764,54535,74,GALAXY
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,0.455040,6751,56368,470,GALAXY


In [3]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder for 'class'
df['class'] = LabelEncoder().fit_transform(df['class'])
df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,redshift,plate,MJD,fiber_ID,class
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,0.634794,5812,56354,171,0
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,0.779136,10445,58158,427,0
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,0.644195,4576,55592,299,0
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,0.932346,9149,58039,775,0
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,0.116123,6121,56187,842,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,0.000000,9374,57749,438,0
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,0.404895,7626,56934,866,0
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,0.143366,2764,54535,74,0
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,0.455040,6751,56368,470,0


**Features are separated from the Target Variables without taking the last row. This with the purpose of making a prediction using those Observations and analyzing the result later.**

In [4]:
# Features
x = df.drop(columns=['class'])[:-1]
x

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,1.237663e+18,317.246996,-0.682254,20.96526,19.81625,19.34186,19.14711,19.05790,4187,301,2,64,1.154061e+18,0.175206,1025,53239,51
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,0.000000,9374,57749,438
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,0.404895,7626,56934,866
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,0.143366,2764,54535,74


In [5]:
# Target Variables
y = df['class'][:-1]
y

0        0
1        0
2        0
3        0
4        0
        ..
99994    0
99995    0
99996    0
99997    0
99998    0
Name: class, Length: 99999, dtype: int32

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Data preprocessing
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Splitting Data
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=0)

# Number of Nearest Neighbors
k = 5

# Training
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(x_train, y_train)

# Predictions
y_pred = knn.predict(x_test)

# Evaluation
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92     11915
           1       0.96      0.81      0.88      3823
           2       0.91      0.79      0.85      4262

    accuracy                           0.90     20000
   macro avg       0.92      0.86      0.88     20000
weighted avg       0.90      0.90      0.90     20000



In [7]:
# Preparing Features
x_predict = df.iloc[-1, :-1]
x_predict

obj_ID         1.237661e+18
alpha          1.968961e+02
delta          4.946464e+01
u              2.262171e+01
g              2.179745e+01
r              2.060115e+01
i              2.000959e+01
z              1.928075e+01
run_ID         3.650000e+03
rerun_ID       3.010000e+02
cam_col        4.000000e+00
field_ID       6.000000e+01
spec_obj_ID    8.343152e+18
redshift       5.429442e-01
plate          7.410000e+03
MJD            5.710400e+04
fiber_ID       8.510000e+02
Name: 99999, dtype: float64

In [8]:
# Standardizing
x_predict = x_predict.values.reshape(1, -1)
x_predict_scaled = scaler.transform(x_predict)

# Fit the LabelEncoder to the original class labels
label_encoder = LabelEncoder()
label_encoder.fit(df['class'])

# Make a prediction using the KNN classifier
prediction = knn.predict(x_predict_scaled)

# Inverse transform the predicted class
predicted_class = label_encoder.inverse_transform(prediction)

# Printing Value Predicted
print("Predicted Class:", predicted_class)


Predicted Class: [0]


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
# Printing Accurate Value to Predict
y_predicted = df.iloc[-1, -1]
print("Actual Predicted Class:", y_predicted)

Actual Predicted Class: 0


It is possible to see how the implementation of this algorithm managed to solve the classification problem by correctly predicting the class with respect to the introduced Features. The hyperparameters used were k = 5 for determining Nearest Neighbors, a Train Size of 80%, a Test Size of 20%, and a Random State = 0 to ensure randomness in training.

## IV.	Loss Function and Optimization Function Identification

In a approach of classification such as this example, a loss function quantifies how well a machine learning model's predictions align with the actual class labels. It measures the degree of error between predicted and true values. Common loss functions for classification include cross-entropy loss and hinge loss.

Furthermore, optimization function, or optimizer, is used for minimizing the loss function. Its purpose is to adjust the model's parameters during training to find the optimal values that reduce the loss and improving the model's ability to make accurate predictions. Common optimizers include Gradient Descent and its variations, such as Stochastic Gradient Descent (SGD) or Adam.

Nevertheless, KNN algorithm operates without the explicit optimization of any objective function since it relies on similarity metrics, like the Euclidean distance, to make predictions.

## V. Bibliography

K-Nearest Neighbor(KNN) algorithm for Machine Learning - Javatpoint. www.javatpoint.com. (n.d.). https://www.javatpoint.com/k-nearest-neighbor-algorithm-for-machine-learning 

¿Qué es knn?. IBM. (n.d.). https://www.ibm.com/mx-es/topics/knn 

Sulla-Torres, Jose & Niebles-Mamani, Luis & Velarde, Rodrigo. (2017). Predicción de incumplimiento de pago de clientes de tarjetas de crédito, con aplicación del algoritmo del k-vecino más cercano y Clas- FriedmanAligned-ST. 10.18687/LACCEI2017.1.1.329. 

fedesoriano. (January 2022). Stellar Classification Dataset - SDSS17. Retrieved [Date Retrieved] from https://www.kaggle.com/fedesoriano/stellar-classification-dataset-sdss17

Terven, Juan & Cordova-Esparza, Diana-Margarita & Ramirez-Pedraza, Alfonzo & Chavez-Urbiola, Edgar. (2023). Loss Functions and Metrics in Deep Learning. A Review. 